In [ ]:
import nltk
from nltk.corpus import brown
from nltk.stem import WordNetLemmatizer
from collections import defaultdict

nltk.download('brown')
nltk.download('wordnet')

In [2]:
lemmatizer = WordNetLemmatizer()
brown_to_custom = {
    'CS': 'CST',   # Potentially CST or CJT, to be determined by context
    'DT': 'DT',    # Determiner
    'WDT': 'WPR',  # Relative pronoun
    'QL': 'RB',    # Adverb
    'CS-HL': 'CST',
    'CS-NC': 'CST',
    'DT-HL': 'DT',
    'DT-NC': 'DT',
    'DT-TL': 'DT',
    'WPS': 'WPR',
    'WPS-HL': 'WPR',
    'WPS-NC': 'WPR',
    'WPS-TL': 'WPR'
}

def is_verb_tag(tag):
    return tag.startswith('V') or tag in ['BE', 'HV', 'DO']

def is_noun_tag(tag):
    return tag.startswith('N') or tag in ['PPS', 'PPO']

def prepare_brown_corpus_for_that(output_file):
    training_data = []
    category_counts = defaultdict(int)
    total_that_count = 0

    for sent in brown.tagged_sents():
        sentence_tokens = []
        contains_relevant_that = False
        for i, (word, tag) in enumerate(sent):
            lemma = lemmatizer.lemmatize(word.lower())
            custom_tag = brown_to_custom.get(tag, tag)
            
            if custom_tag == 'CST':  # This includes 'CS', 'CS-HL', 'CS-NC'
                prev_tag = sent[i-1][1] if i > 0 else ''
                if is_verb_tag(prev_tag):
                    custom_tag = 'CJT'
                # If it's not a verb, it remains 'CST'
            
            if word.lower() == 'that' and custom_tag in ['WPR', 'CST', 'CJT', 'DT', 'RB']:
                category_counts[custom_tag] += 1
                total_that_count += 1
                contains_relevant_that = True
            
            sentence_tokens.append(f"{word}\t{custom_tag}\t{lemma}")
        
        if contains_relevant_that:
            training_data.extend(sentence_tokens)
            training_data.append("")

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("\n".join(training_data))

    print(f"Corpus d'entraînement sauvegardé dans : {output_file}")
    print(f"Nombre total d'occurrences de 'that': {total_that_count}")
    print("Nombre d'occurrences de 'that' pour chaque catégorie :")
    for category, count in category_counts.items():
        print(f"{category}: {count}")

output_file = "corpus_brown_that_custom.txt"
prepare_brown_corpus_for_that(output_file)

Corpus d'entraînement sauvegardé dans : corpus_brown_that_custom.txt
Nombre total d'occurrences de 'that': 10457
Nombre d'occurrences de 'that' pour chaque catégorie :
CST: 3831
CJT: 2636
WPR: 1662
DT: 2272
RB: 56


In [3]:
from collections import defaultdict

def create_lexicon_and_tag_files(input_file, lexicon_file, tag_file):
    word_tag_lemma = defaultdict(set)
    all_tags = set()
    
    with open(input_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            if line.strip():
                word, tag, lemma = line.strip().split('\t')
                word_tag_lemma[word.lower()].add((tag, lemma))
                all_tags.add(tag)
    
    # Écriture du fichier lexicon
    with open(lexicon_file, 'w', encoding='utf-8') as outfile:
        for word, tag_lemmas in word_tag_lemma.items():
            outfile.write(f"{word}\t" + "\t".join(f"{tag} {lemma}" for tag, lemma in tag_lemmas) + "\n")
    
    # Écriture du fichier tag
    with open(tag_file, 'w', encoding='utf-8') as outfile:
        for tag in sorted(all_tags):
            outfile.write(f"{tag}\n")

    print(f"Fichier lexicon créé : {lexicon_file}")
    print(f"Fichier tag créé : {tag_file}")

# Utilisation de la fonction
input_file = "corpus_brown_that_custom.txt"
lexicon_file = "lexicon_brown.txt"
tag_file = "tags_brown.txt"

create_lexicon_and_tag_files(input_file, lexicon_file, tag_file)

Fichier lexicon créé : lexicon_brown.txt
Fichier tag créé : tags_brown.txt


In [6]:
import subprocess
import os

# Définir les chemins
tagger_bin = os.path.expanduser("~/treetagger/bin/train-tree-tagger")  # Adapter selon l'OS
lexicon_file = "lexicon_brown.txt"
open_class_file = "tags_brown.txt"
train_file = "corpus_brown_that_custom.txt"
output_model = "english_brown_model.par"

# Vérifier que les fichiers existent
for file in [tagger_bin, lexicon_file, open_class_file, train_file]:
    if not os.path.exists(file):
        print(f"Erreur : Fichier introuvable -> {file}")
        exit(1)

# Commande correcte pour entraîner TreeTagger
train_command = [
    tagger_bin, "-utf8",
    "-st", ".",
    lexicon_file, open_class_file, train_file, output_model
]

# Exécuter l'entraînement
process = subprocess.run(train_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Afficher la sortie
print(process.stdout.decode())
print(process.stderr.decode())

# Vérifier si le fichier modèle a été généré
if os.path.exists(output_model):
    print(f"✅ Modèle entraîné avec succès : {output_model}")
else:
    print(f"❌ Erreur : le modèle n'a pas été généré.")



train-tree-tagger -cl 2 -dtg 1.00 -sw 1.00 -ecw 1.00 -stg 1.00 -ptg -1.00 lexicon_brown.txt tags_brown.txt corpus_brown_that_custom.txt english_brown_model.par

	reading the lexicon ...
		reading the tagset ...
		reading the lemmas ...
		reading the entries ...
		sorting the lexicon ...
		reading the open class tags ...
	calculating tag frequencies ...
247000	making affix tree ...
prefix lexicon: 15945 nodes
suffix lexicon: 3120 nodes
	reading classes ...
	making ngram table ...
279449	29083
finished.
	making decision tree ...
58	saving parameters ...

Number of nodes: 59
Max. path length: 24

done.

✅ Modèle entraîné avec succès : english_brown_model.par


In [7]:
import os
import treetaggerwrapper

# Définir le chemin vers TreeTagger
tagger_dir = os.path.expanduser("~/treetagger/bin/train-tree-tagger")  # Remplacez par le chemin réel de votre installation TreeTagger
os.environ['TAGDIR'] = tagger_dir


# Initialiser les taggers

tagger_brown = treetaggerwrapper.TreeTagger(TAGLANG='en',TAGDIR=tagger_dir, TAGPARFILE="english_brown_model.par")


# Fonction pour calculer la précision
#Cette ligne définit la fonction. Elle prend deux paramètres :

    #tagged_sentences : Une liste de phrases étiquetées par TreeTagger. Chaque phrase est une liste de tuples (mot, tag, lemme).
    #expected_tags : Une liste des tags corrects attendus pour 'that' dans cette catégorie grammaticale.

def calculate_precision(tagged_sentences, expected_tags):
    correct = sum(1 for sent in tagged_sentences for word, tag, _ in sent
                  if word.lower() == 'that' and tag in expected_tags)
    total = sum(1 for sent in tagged_sentences for word, _, _ in sent
                if word.lower() == 'that')
    return correct / total if total > 0 else 0

# Fichiers de test et leurs catégories correspondantes
test_files = {
    'that_adv.txt': 'RB',
    'that_conjunction.txt': ['CST','CJT'],
    'that_determiner.txt': 'DT',
    'that_pronoun.txt': 'WPR'
}
results = {}

for file, tags in test_files.items():
    with open(file, 'r') as f:
        sentences = f.readlines()

    
    tagged_brown = [tagger_brown.tag_text(sent) for sent in sentences]
    parsed_brown = [treetaggerwrapper.make_tags(tag) for tag in tagged_brown]


    # Calculer les précisions
    precision_brown = calculate_precision(parsed_brown, tags)

    results[file] = {'Brown': precision_brown}

# Afficher les résultats
print("| Catégorie de 'that' | Précision Brown (english_brown_model.par) |")
print("|---------------------|-------------------------|")
for file, precisions in results.items():
    category = file.split('.')[0].split('_')[1].capitalize()
    print(f"| {category:<19} | {precisions['Brown']:.2%}{' ':>17} |")

TreeTaggerError: Bad TreeTagger directory: /Users/diamouserignetoubandiaye/treetagger/bin/train-tree-tagger